### 네이버 영화 리뷰 감성 분석

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')

In [3]:
print(train_df.shape, test_df.shape)

(150000, 3) (50000, 3)


In [4]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### 1. 데이터 전처리

- train dataset

In [5]:
# 결측치 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [6]:
# 결측치 데이터 삭제
train_df.dropna(how='any', inplace=True)
train_df.shape

(149995, 3)

In [7]:
# 중복 데이터 확인
train_df.document.nunique()

146182

In [8]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)

In [9]:
train_df.shape

(146182, 3)

In [10]:
# 데이터 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터

In [11]:
test_df.dropna(how='any', inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [12]:
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

In [13]:
train_df = train_df.iloc[:, 1:]
train_df.head()

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [14]:
test_df = test_df.iloc[:, 1:]
test_df.head()

,document,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


### 2. 텍스트 전처리

- 트레인 데이터 셋

In [15]:
# 한글 이외의 문자는 공백으로 처리, strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()
train_df.head()

,document,label
0,아 더빙 진짜 짜증나네요 목소리,0
1,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [40]:
# 영어 데이터 --> ''만 남게됨
train_df.document.apply(lambda x: len(x)).sort_values().value_counts()

14     4143
13     4141
12     4040
15     4037
16     3779
       ... 
119     136
115     132
117     131
141      11
142      11
Name: document, Length: 142, dtype: int64

In [36]:
train_df.document.replace('', np.nan, inplace=True)
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 2)

- 테스트 데이터셋

In [70]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()

In [71]:
test_df.document.replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48852, 2)

In [72]:
train_df.to_csv('data/네이버영화리뷰_전처리_train.tsv', sep='\t', index=False)
test_df.to_csv('data/네이버영화리뷰_전처리_test.tsv', sep='\t', index=False)

### 3. 한글처리

In [41]:
from konlpy.tag import Okt
okt = Okt()

In [42]:
test = '흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나'
okt.morphs(test)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍지', '않구나']

In [43]:
okt.morphs(test, stem=True)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']

In [57]:
stopwords = pd.read_csv('data/Korean_stopwords.txt', sep='\t')

In [46]:
with open('data/Korean_stopwords.txt') as f:
    lines = f.readlines()
len(lines)

100

In [52]:
stop_words = [line.split('\t')[0] for line in lines]
stop_words[:10]

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보']

- 형태소 분석기를 돌려서 나온 결과에서 stop_words 제거하기

In [65]:
morphs = okt.morphs(test, stem=True)
clean_morphs = [word for word in morphs if word not in stop_words]

In [62]:
morphs

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']

In [63]:
# clean_morphs를 문자열로 만들어야 CountVectorizer의 입력으로 사용 가능
clean_data = ' '.join(clean_morphs)
clean_data

'흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다'

- train / test dataset 적용

In [67]:
from tqdm.notebook import tqdm

X_train = []
for review in tqdm(train_df.document):
    morphs = okt.morphs(review, stem=True)
    clean_morphs = [word for word in morphs if word not in stop_words]
    clean_data = ' '.join(clean_morphs)
    X_train.append(clean_data)
X_train[:5]

  0%|          | 0/145393 [00:00<?, ?it/s]

['아 더빙 진짜 짜증나다 목소리',
 '흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다',
 '너 무재 밓었 다그 래서 보다 추천 다',
 '교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정',
 '사이 몬페 의 익살스럽다 연기 돋보이다 영화 스파이더맨 에서 늙다 보이다 하다 커스틴 던스트 너무나도 이쁘다 보이다']

In [73]:
%%time
X_test = []
for review in test_df.document:
    morphs = okt.morphs(review, stem=True)
    clean_morphs = [word for word in morphs if word not in stop_words]
    clean_data = ' '.join(clean_morphs)
    X_test.append(clean_data)
X_test[:5]

CPU times: total: 2min 18s
Wall time: 2min 22s


['굳다 ㅋ',
 '뭐 야 평점 은 나쁘다 않다 짜다 리 는 더욱 아니다',
 '지루하다 않다 완전 막장 임 돈 주다 보기 에는',
 '만 아니다 별 다섯 주다 왜 로 나오다 제 심기 를 불편하다 하다',
 '음악 주가 되다 최고 의 음악 영화']

In [74]:
y_train = train_df.label.values
y_test = test_df.label.values

### 4. 피쳐변환 + 모델학습/평가 

In [75]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [77]:
pipeline = Pipeline([
    ('cvect', CountVectorizer()),
    ('nb', MultinomialNB())
])
%time pipeline.fit(X_train, y_train)

CPU times: total: 2.06 s
Wall time: 2.05 s


Pipeline(steps=[('cvect', CountVectorizer()), ('nb', MultinomialNB())])

In [78]:
pipeline.score(X_test, y_test)

0.827581265864243

### 5. 실제 데이터로 검증

In [79]:
reviews = [
    '이 영화 개꿀잼 ㅋㅋㅋ',
    '이 영화 핵노잼 ㅠㅠ',
    '이딴게 영화냐 ㅉㅉ',
    '감독 뭐하는 놈이냐?',
    '와 개쩐다 정말 세계관 최강자들의 영화다'
]

In [80]:
# 전처리 - 한글이외의 문자 blank 변환
import re
reviews = list(map(lambda s: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', s), reviews))

In [81]:
reviews

['이 영화 개꿀잼 ㅋㅋㅋ',
 '이 영화 핵노잼 ㅠㅠ',
 '이딴게 영화냐 ㅉㅉ',
 '감독 뭐하는 놈이냐 ',
 '와 개쩐다 정말 세계관 최강자들의 영화다']

In [82]:
# 형태소 분석 후 불용어 제거
processed_reviews = []
for review in reviews:
    morphs = okt.morphs(review, stem=True)
    clean_morphs = [word for word in morphs if word not in stop_words]
    clean_data = ' '.join(clean_morphs)
    processed_reviews.append(clean_data)

In [83]:
pipeline.predict(processed_reviews)

array([1, 0, 0, 0, 1], dtype=int64)

In [84]:
pipeline.predict_proba(processed_reviews)

array([[0.06260368, 0.93739632],
       [0.91612638, 0.08387362],
       [0.9906787 , 0.0093213 ],
       [0.9297786 , 0.0702214 ],
       [0.09745728, 0.90254272]])

### 6. 최적 파라미터 찾기

In [85]:
from sklearn.model_selection import GridSearchCV
params = {
    'cvect__ngram_range': [(1,1), (1,2)],
    'cvect__max_df': [0.95, 0.98]
}
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 48 s
Wall time: 47 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('cvect', CountVectorizer()),
                                       ('nb', MultinomialNB())]),
             param_grid={'cvect__max_df': [0.95, 0.98],
                         'cvect__ngram_range': [(1, 1), (1, 2)]},
             scoring='accuracy')

In [86]:
grid_pipe.best_params_

{'cvect__max_df': 0.95, 'cvect__ngram_range': (1, 2)}

In [87]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.843936788667813

- 다음 작업을 위한 준비

In [88]:
df = pd.DataFrame({'x':X_train, 'y':y_train})
df.head()

,x,y
0,아 더빙 진짜 짜증나다 목소리,0
1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다,1
2,너 무재 밓었 다그 래서 보다 추천 다,0
3,교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정,0
4,사이 몬페 의 익살스럽다 연기 돋보이다 영화 스파이더맨 에서 늙다 보이다 하다 커스...,1


In [89]:
df.to_csv('data/네이버영화리뷰_train_형태소분석완료.csv', index=False)